In [42]:
import requests
from requests import Response
# import json


def get_response() -> Response | None:
    url_path:str = 'https://lisp.land.moi.gov.tw/MoiMMSv2/SectionList.ashx'

    print(f'開始下載:')

    try:
        response:Response = requests.get(url_path, timeout=15)
        response.raise_for_status()
    except Exception as e:
        print('連線失敗。')
        print(e)
        return None
    
    if response.status_code != 200:
        print('下載失敗。')
        print(response.reason)
        return None
    
    print('下載成功')
    return response

def generateCityTownMaps(data:list[dict]) -> dict:
    maps = {}

    for item in data:
        city = item['CITY']
        town = item['TOWN']

        if(city in maps):
            maps[city].append(town)
        else:
            maps[city] = [town]

    return maps


rawData:Response | None = get_response()
raw:list[dict] = rawData.json()

# with open('SectionList.ashx.json') as file:
#     raw = json.load(file)
#     # display(raw)

maps = generateCityTownMaps(raw)

display(maps)

開始下載:
下載成功


{'臺北市': ['中正區',
  '文山區',
  '中正區',
  '萬華區',
  '大同區',
  '中山區',
  '內湖區',
  '松山區',
  '南港區',
  '信義區',
  '士林區',
  '北投區',
  '大安區'],
 '臺中市': ['中區',
  '東區',
  '南區',
  '西區',
  '北區',
  '北屯區',
  '西屯區',
  '南屯區',
  '豐原區',
  '后里區',
  '神岡區',
  '大甲區',
  '外埔區',
  '大安區',
  '清水區',
  '沙鹿區',
  '梧棲區',
  '東勢區',
  '新社區',
  '石岡區',
  '和平區',
  '潭子區',
  '大雅區',
  '烏日區',
  '霧峰區',
  '大里區',
  '太平區',
  '大肚區',
  '龍井區'],
 '基隆市': ['中正區', '七堵區', '暖暖區', '仁愛區', '中山區', '安樂區', '信義區'],
 '台南市': ['北區',
  '安平區',
  '中西區',
  '安南區',
  '東區',
  '南區',
  '新營區',
  '鹽水區',
  '柳營區',
  '白河區',
  '後壁區',
  '東山區',
  '麻豆區',
  '下營區',
  '六甲區',
  '官田區',
  '大內區',
  '佳里區',
  '西港區',
  '七股區',
  '將軍區',
  '北門區',
  '學甲區',
  '新化區',
  '善化區',
  '新市區',
  '安定區',
  '山上區',
  '左鎮區',
  '仁德區',
  '歸仁區',
  '關廟區',
  '龍崎區',
  '玉井區',
  '楠西區',
  '南化區',
  '永康區'],
 '高雄市': ['鹽埕區',
  '鼓山區',
  '前金區',
  '旗津區',
  '新興區',
  '苓雅區',
  '前鎮區',
  '小港區',
  '三民區',
  '左營區',
  '楠梓區',
  '岡山區',
  '橋頭區',
  '燕巢區',
  '永安區',
  '彌陀區',
  '梓官區',
  '鳳山區',
  '大樹區',
  '旗山區',
  '甲仙區',
  '杉林區',
  '內門區',


In [51]:
import pandas as pd
from pandas import Series

def splitCityTown(item:Series):
    for city in maps:
        if(item['區域別'].startswith(city)):
            item['縣市'] = city
            tmpTown = item['區域別'][len(city):]
            if(tmpTown in maps[city]):
                item['鄉鎮區'] = tmpTown

    if('縣市' not in item.keys()):
        item['縣市'] = None

    if('鄉鎮區' not in item.keys()):
        item['鄉鎮區'] = None

    return item

df1 = pd.read_csv('各鄉鎮市區人口密度.csv')
df1.columns = df1.iloc[0]
df2 = df1.drop(index=0)
df3 = df2.dropna()

df4 = df3.apply(splitCityTown, axis=1)
df4

,統計年,區域別,年底人口數,土地面積,人口密度,縣市,鄉鎮區
1,106,新北市板橋區,551480,23.1373,23835,新北市,板橋區
2,106,新北市三重區,387484,16.317,23747,新北市,三重區
3,106,新北市中和區,413590,20.144,20532,新北市,中和區
4,106,新北市永和區,222585,5.7138,38956,新北市,永和區
5,106,新北市新莊區,416524,19.7383,21102,新北市,新莊區
...,...,...,...,...,...,...,...
366,106,連江縣北竿鄉,2360,9.9,238,連江縣,北竿鄉
367,106,連江縣莒光鄉,1624,4.7,346,連江縣,莒光鄉
368,106,連江縣東引鄉,1352,3.8,356,連江縣,東引鄉
369,106,東沙群島,…,2.38,…,None,None


In [96]:
import requests
from requests import Response
import pandas as pd
from pandas import Series

# 政府資料標準平台的土地段名代碼（ＡＰＩ）
# https://schema.gov.tw/code/detail/122674
def get_response() -> Response | None:
    url_path:str = 'https://lisp.land.moi.gov.tw/MoiMMSv2/SectionList.ashx'

    print(f'開始下載:')

    try:
        response:Response = requests.get(url_path, timeout=15)
        response.raise_for_status()
    except Exception as e:
        print('連線失敗。')
        print(e)
        return None
    
    if response.status_code != 200:
        print('下載失敗。')
        print(response.reason)
        return None
    
    print('下載成功')
    return response

def generateCityTownMaps(data:list[dict]) -> dict[list]:
    maps = {}

    for item in data:
        city = item['CITY']
        town = item['TOWN']

        if(city in maps):
            maps[city].append(town)
        else:
            maps[city] = [town]

    return maps

def splitCityTown(item:Series, maps:dict[list]) -> tuple[str | None, str | None]:
    tmpCity = None
    tmpTown = None

    for city in maps:
        if(item['區域別'].startswith(city)):
            tmpCity = city
            splittedTown = item['區域別'][len(city):]
            if(splittedTown in maps[city]):
                tmpTown = splittedTown

    return tmpCity, tmpTown

def main():
    rawData:Response | None = get_response()
    if(rawData is None):
        return
    
    raw:list[dict] = rawData.json()

    maps:dict[list] = generateCityTownMaps(raw)

    df1 = pd.read_csv('各鄉鎮市區人口密度.csv')
    df1.columns = df1.iloc[0]
    df2 = df1.drop(index=0)
    df3 = df2.dropna()
    # 為了避免 A value is trying to be set on a copy of a slice from a DataFrame 等之類的警告
    # 另外複製出來而不直接把['縣市','鄉鎮區']塞入原來的df3
    df4 = df3.copy()

    df4[['縣市','鄉鎮區']] = df3.apply(lambda s: splitCityTown(s, maps), axis=1, result_type='expand')

    display(df4)

main()

開始下載:
下載成功


,統計年,區域別,年底人口數,土地面積,人口密度,縣市,鄉鎮區
1,106,新北市板橋區,551480,23.1373,23835,新北市,板橋區
2,106,新北市三重區,387484,16.317,23747,新北市,三重區
3,106,新北市中和區,413590,20.144,20532,新北市,中和區
4,106,新北市永和區,222585,5.7138,38956,新北市,永和區
5,106,新北市新莊區,416524,19.7383,21102,新北市,新莊區
...,...,...,...,...,...,...,...
366,106,連江縣北竿鄉,2360,9.9,238,連江縣,北竿鄉
367,106,連江縣莒光鄉,1624,4.7,346,連江縣,莒光鄉
368,106,連江縣東引鄉,1352,3.8,356,連江縣,東引鄉
369,106,東沙群島,…,2.38,…,None,None
